### 0. Libraries and utilities

In [1]:
import re
import unicodedata
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

C:\Users\A0860770\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\A0860770\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove accents
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    tokens = text.split()

    # Remove English stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Join the tokens back into a single string
    text = ' '.join(tokens)

    return text

# Limpiamos el texto
preprocess_text('What can I say, I love this place')

'say love place'

### 1. Preprocessing

In [3]:
data_path = r'Data\Gungor_2018_VictorianAuthorAttribution_data-train.csv'
df = pd.read_csv(data_path, encoding='latin-1')

# ## URL from github repo, load as dataframe
# url = 'https://raw.githubusercontent.com/ccsarmientot/text_author_classifier/master/datasets/sample_victorian.parquet'
# url = 'datasets/sample_victorian.parquet'
# df = pd.read_parquet(url)

print(f'Shape of dataframe: {df.shape}')
df.head(5)

Shape of dataframe: (53678, 2)


,text,author
0,ou have time to listen i will give you the ent...,1
1,wish for solitude he was twenty years of age a...,1
2,and the skirt blew in perfect freedom about th...,1
3,of san and the rows of shops opposite impresse...,1
4,an hour s walk was as tiresome as three in a s...,1


In [4]:
avg_chr = np.mean(df['text'].apply(len))
f'Cantidad promedio de caracteres por texto: {avg_chr:,.2f}'

'Cantidad promedio de caracteres por texto: 4,942.97'

In [5]:
avg_chr = np.mean(df['text'].apply(lambda x: len(x.split(' '))))
f'Cantidad promedio de palabras por texto: {avg_chr:,.2f}'

'Cantidad promedio de palabras por texto: 1,001.00'

In [6]:
## Se identifica un desbalance de clases:
df['author'].value_counts()

author
8     6914
26    4441
14    2696
37    2387
45    2312
21    2307
39    2266
48    1825
33    1742
19    1543
4     1483
15    1460
43    1266
38    1163
25    1159
9     1108
18    1078
42    1022
30     972
50     914
1      912
41     911
28     823
10     755
32     703
36     693
17     660
35     659
29     645
12     627
46     605
20     587
22     495
13     485
44     468
23     455
34     453
40     430
6      407
11     383
2      382
24     380
27     306
3      213
16     183
Name: count, dtype: int64

In [7]:
# ## Getting sample fo 
# df_sample = df.sample(10_000)
# n_authors = df_sample['author'].nunique()
# print(f'Authors in df_sample: {n_authors}')
# df_sample.head(5)

## 2. Modelling

In [8]:
# Importamos librerias
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV

results = []
X = df['text']
y = df['author']

# Dividimos los datos en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
## Se aplica el preprocesameinto

# 1. Feature Extraction
tfidf = TfidfVectorizer(max_features=1000, preprocessor=preprocess_text)
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

# 2. Oversampling
oversample = RandomOverSampler()
X_train, y_train = oversample.fit_resample(X_train, y_train)

### 2.0 Define function to iterate over

In [10]:
def train_cv_models(model_name:str, classifier, 
                    param_grid:dict[list], 
                    X_train, y_train) -> dict:

    # Creacion del pipeline del modelo inicial
    model = Pipeline(steps=[
        ## Se aplica el modelo
        (model_name, classifier)
    ])

    ######################## PRIMERA BÚSQUEDA DE PARÁMETROS ####################
    print(' Primera búsqueda de parámetros '.center(80, '#'))

    # Creamos el objeto GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=2)

    # Ajustamos el modelo a los datos de entrenamiento
    grid_search.fit(X_train, y_train)

    # Obtenemos los mejores parámetros
    best_params = grid_search.best_params_
    print(f'Resultados primera búsqueda: {best_params}', '\n')

    ######################## SEGUNDA BÚSQUEDA DE PARÁMETROS ####################
    print(' Segunda búsqueda de parámetros '.center(80, '#'), '\n')
    

    # A los parámetros que deben ser enteros se resta y suma 1
    int_list = ['min_samples_split', 'max_depth', 'n_neighbors', 'min_samples_leaf', 'n_estimators']
    int_grid = {k:[v, v+1] for k,v in best_params.items() if any([i in k for i in int_list])}
    ## Clean zero values
    int_grid = {k:[c for c in v if c != 0] for k,v in int_grid.items()}

    # A los parámetros numéricos encontrados se resta y suma el 10 %
    int_params_grid = {k:[v-(v/10), v, v+(v/10)] for k,v in best_params.items() if isinstance(v, (float, int))}
    
    # A los parámetros en formato string encontrados se deja el mejor
    str_params_grid = {k:[v] for k,v in best_params.items() if not isinstance(v, list)}

    best_params_grid = {**str_params_grid, **int_params_grid, **int_grid}
    # best_params_grid['tfidf__max_features'] = [1000]

    print('DEBUG: params after transform: ', best_params_grid)

    # Creamos el objeto GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=best_params_grid, cv=5, verbose=2)

    # Ajustamos el modelo a los datos de entrenamiento
    grid_search.fit(X_train, y_train)

    # Obtenemos los mejores parámetros
    best_params = grid_search.best_params_
    print(f'Resultados segunda búsqueda: {best_params}', '\n')

    return best_params


def train_final_model(model_name, classifier, X_train, y_train) -> list:

    print(' Creando modelo final con los mejores parámetros '.center(80, '#'))
    
    # Creacion del pipeline modelo final
    final_model = Pipeline(steps=[
        ## Se aplica el modelo
        (model_name, classifier)
    ])

    # Ajuste del modelo
    final_model.fit(X_train, y_train)

    # Medimos el accuracy del modelo
    accuracy = final_model.score(X_test, y_test)
    print(f'Accuracy de {model_name}: {accuracy:,.2%}')

    # Se predicen las clases para test
    y_pred = final_model.predict(X_test)

    p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    return [model_name, accuracy, p, r, f1]


### 2.1 Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression

# Definimos el modelo a usar
classifier = LogisticRegression()
model_name = 'LogisticRegression'

# Definimos los parámetros a explorar
param_grid = {
    
    f'{model_name}__C': [0.1, 1, 10],
}

best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}

best_classifier = LogisticRegression(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ..........................LogisticRegression__C=0.1; total time=  29.0s
[CV] END ..........................LogisticRegression__C=0.1; total time=  25.0s
[CV] END ..........................LogisticRegression__C=0.1; total time=  26.8s
[CV] END ..........................LogisticRegression__C=0.1; total time=  25.1s
[CV] END ..........................LogisticRegression__C=0.1; total time=  25.8s
[CV] END ............................LogisticRegression__C=1; total time=  52.7s
[CV] END ............................LogisticRegression__C=1; total time=  44.0s
[CV] END ............................LogisticRegression__C=1; total time=  57.8s
[CV] END ............................LogisticRegression__C=1; total time=  49.4s
[CV] END ............................LogisticRegression__C=1; total time=  46.0s
[CV] END ...........................LogisticRegre

In [11]:
from sklearn.linear_model import LogisticRegression

# Definimos los mejores parámetros encontrados
model_best_params = {
    'C': 11,
}
model_name = 'LogisticRegression'
best_classifier = LogisticRegression(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################
Accuracy de LogisticRegression: 92.32%


C:\Users\A0860770\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 2.2 Naive Bayes

In [12]:
from sklearn.naive_bayes import MultinomialNB

# Definimos el modelo a usar
classifier = MultinomialNB()
model_name = 'NaiveBayes'

# Definimos los parámetros a explorar
param_grid = {
    
    f'{model_name}__alpha': [0.1, 1, 10],
}


best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}

best_classifier = MultinomialNB(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ..............................NaiveBayes__alpha=0.1; total time=   0.4s
[CV] END ..............................NaiveBayes__alpha=0.1; total time=   0.5s
[CV] END ..............................NaiveBayes__alpha=0.1; total time=   0.4s
[CV] END ..............................NaiveBayes__alpha=0.1; total time=   0.4s
[CV] END ..............................NaiveBayes__alpha=0.1; total time=   0.4s
[CV] END ................................NaiveBayes__alpha=1; total time=   0.4s
[CV] END ................................NaiveBayes__alpha=1; total time=   0.4s
[CV] END ................................NaiveBayes__alpha=1; total time=   0.4s
[CV] END ................................NaiveBayes__alpha=1; total time=   0.4s
[CV] END ................................NaiveBayes__alpha=1; total time=   0.4s
[CV] END ...............................NaiveBaye

In [12]:
from sklearn.naive_bayes import MultinomialNB

# Definimos los mejores parámetros encontrados
model_best_params = {
    'alpha': 0.9,
}
model_name = 'NaiveBayes'
best_classifier = MultinomialNB(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################
Accuracy de NaiveBayes: 78.76%


### 2.3 KNeighbors

In [13]:
from sklearn.neighbors import KNeighborsClassifier

# Definimos el modelo a usar
classifier = KNeighborsClassifier()
model_name = 'KNeighbors'

# Definimos los parámetros a explorar
param_grid = {
    
    f'{model_name}__n_neighbors': [3, 5],
    f'{model_name}__metric': ['euclidean', 'manhattan', 'cosine']
}


best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}
best_classifier = KNeighborsClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=3; total time= 2.0min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=3; total time= 1.7min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=3; total time= 1.7min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=3; total time= 1.8min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=3; total time= 1.8min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=5; total time= 1.9min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=5; total time= 1.7min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=5; total time= 1.7min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=5; total time= 1.8min
[CV] END KNeighbors__metric=euclidean, KNeighbors__n_neighbors=5; total time= 1.7min
[CV] END 

In [13]:
from sklearn.neighbors import KNeighborsClassifier

# Definimos los mejores parámetros encontrados
model_best_params = {
    'metric': 'euclidean',
    'n_neighbors': 3
}
model_name = 'KNeighbors'
best_classifier = KNeighborsClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################


### 2.4 Arboles de decisión

In [14]:
from sklearn.tree import DecisionTreeClassifier

# Definimos el modelo a usar
classifier = DecisionTreeClassifier()
model_name = 'DecisionTree'

# Definimos los parámetros a explorar
param_grid = {
    
    f'{model_name}__criterion': ['gini', 'entropy'],
    f'{model_name}__max_depth': [5, 10],
    f'{model_name}__min_samples_split': [5, 10],
    f'{model_name}__min_samples_leaf': [2, 3]
}


best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}

best_classifier = DecisionTreeClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END DecisionTree__criterion=gini, DecisionTree__max_depth=5, DecisionTree__min_samples_leaf=2, DecisionTree__min_samples_split=5; total time=  11.7s
[CV] END DecisionTree__criterion=gini, DecisionTree__max_depth=5, DecisionTree__min_samples_leaf=2, DecisionTree__min_samples_split=5; total time=  12.0s
[CV] END DecisionTree__criterion=gini, DecisionTree__max_depth=5, DecisionTree__min_samples_leaf=2, DecisionTree__min_samples_split=5; total time=  11.2s
[CV] END DecisionTree__criterion=gini, DecisionTree__max_depth=5, DecisionTree__min_samples_leaf=2, DecisionTree__min_samples_split=5; total time=  11.2s
[CV] END DecisionTree__criterion=gini, DecisionTree__max_depth=5, DecisionTree__min_samples_leaf=2, DecisionTree__min_samples_split=5; total time=  11.0s
[CV] END DecisionTree__criterion=gini, DecisionTree__max_depth=5, DecisionTree__min_samp

: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Definimos los mejores parámetros encontrados
model_best_params = {
    'criterion': 'entropy',
    'max_depth': 11,
    'min_samples_leaf': 2
}
model_name = 'DecisionTree'
best_classifier = DecisionTreeClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################
Accuracy de DecisionTree: 18.85%


### 2.5 Support Vector Machine

In [ ]:
from sklearn.svm import SVC

# Definimos el modelo a usar
classifier = SVC()
model_name = 'SVC'

# Definimos los parámetros a explorar
param_grid = {

    f'{model_name}__C': [1, 10],
    f'{model_name}__kernel': ['linear', 'rbf'],
    f'{model_name}__gamma': [0.1, 1]
}


best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}

best_classifier = SVC(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] END .......SVC__C=1, SVC__gamma=0.1, SVC__kernel=linear; total time=13.9min
[CV] END .......SVC__C=1, SVC__gamma=0.1, SVC__kernel=linear; total time=14.7min
[CV] END ..........SVC__C=1, SVC__gamma=0.1, SVC__kernel=rbf; total time=32.6min
[CV] END ..........SVC__C=1, SVC__gamma=0.1, SVC__kernel=rbf; total time=33.7min
[CV] END .........SVC__C=1, SVC__gamma=1, SVC__kernel=linear; total time=13.7min
[CV] END .........SVC__C=1, SVC__gamma=1, SVC__kernel=linear; total time=14.8min
[CV] END ............SVC__C=1, SVC__gamma=1, SVC__kernel=rbf; total time=16.9min
[CV] END ............SVC__C=1, SVC__gamma=1, SVC__kernel=rbf; total time=18.1min
[CV] END ......SVC__C=10, SVC__gamma=0.1, SVC__kernel=linear; total time=10.2min
[CV] END ......SVC__C=10, SVC__gamma=0.1, SVC__kernel=linear; total time=11.1min
[CV] END .........SVC__C=10, SVC__gamma=0.1, SVC_

In [ ]:
from sklearn.svm import SVC

# Definimos los mejores parámetros encontrados
model_best_params = {
    'C': 10,
    'gamma': 1, 
    'kernel': 'rbf'
    }
model_name = 'SVC'
best_classifier = SVC(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################
Accuracy de SVC: 84.25%


C:\Users\A0860770\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2.6 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Definimos el modelo a usar
classifier = RandomForestClassifier()
model_name = 'RandomForest'

# Definimos los parámetros a explorar
param_grid = {

    f'{model_name}__n_estimators': [100, 200],
    f'{model_name}__max_depth': [5, 10],
    f'{model_name}__min_samples_split': [2, 5],
    f'{model_name}__min_samples_leaf': [2, 4]
}


best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}

best_classifier = RandomForestClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV] END RandomForest__max_depth=5, RandomForest__min_samples_leaf=2, RandomForest__min_samples_split=2, RandomForest__n_estimators=100; total time=  10.0s
[CV] END RandomForest__max_depth=5, RandomForest__min_samples_leaf=2, RandomForest__min_samples_split=2, RandomForest__n_estimators=100; total time=   9.8s
[CV] END RandomForest__max_depth=5, RandomForest__min_samples_leaf=2, RandomForest__min_samples_split=2, RandomForest__n_estimators=200; total time=  18.5s
[CV] END RandomForest__max_depth=5, RandomForest__min_samples_leaf=2, RandomForest__min_samples_split=2, RandomForest__n_estimators=200; total time=  19.2s
[CV] END RandomForest__max_depth=5, RandomForest__min_samples_leaf=2, RandomForest__min_samples_split=5, RandomForest__n_estimators=100; total time=  10.0s
[CV] END RandomForest__max_depth=5, RandomForest__min_samples_leaf=2, RandomFo

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
126 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/imblearn/pipeline.py", line 333, in fit
    self._final_estimator.fit(Xt, yt, **last_step_params["fit"])
  File "/usr/local/lib/

Resultados segunda búsqueda: {'RandomForest__max_depth': 11, 'RandomForest__min_samples_leaf': 1, 'RandomForest__min_samples_split': 2, 'RandomForest__n_estimators': 200} 

############### Creando modelo final con los mejores parámetros ################
Accuracy de RandomForest: 57.40%


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Definimos los mejores parámetros encontrados
model_best_params = {
    'n_estimators': 200,
    'max_depth': 11,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}
model_name = 'RandomForest'
best_classifier = RandomForestClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################
Accuracy de RandomForest: 55.70%


### 2.7 MLP Classifier (Multi-Layer Perceptron)

In [ ]:
from sklearn.neural_network import MLPClassifier

# Definimos el modelo a usar
classifier = MLPClassifier()
model_name = 'MLP'

# Definimos los parámetros a explorar
param_grid = {
    
    f'{model_name}__hidden_layer_sizes': [(100, 50), (50, 50)],
    f'{model_name}__activation': ['relu', 'tanh'],
    f'{model_name}__solver': ['adam', 'sgd'],
    f'{model_name}__alpha': [0.001, 0.01],
    f'{model_name}__learning_rate': ['constant', 'adaptive']
}

best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}

best_classifier = MLPClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END MLP__activation=relu, MLP__alpha=0.001, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=adam; total time= 4.5min
[CV] END MLP__activation=relu, MLP__alpha=0.001, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=adam; total time= 4.4min
[CV] END MLP__activation=relu, MLP__alpha=0.001, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=adam; total time= 4.9min
[CV] END MLP__activation=relu, MLP__alpha=0.001, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=adam; total time= 4.1min
[CV] END MLP__activation=relu, MLP__alpha=0.001, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=adam; total time= 4.5min


c:\Users\ccsar\miniconda3\envs\env_nlp\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END MLP__activation=relu, MLP__alpha=0.001, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=sgd; total time=10.1min


c:\Users\ccsar\miniconda3\envs\env_nlp\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END MLP__activation=relu, MLP__alpha=0.001, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=sgd; total time= 9.9min


...

[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=sgd; total time=10.1min


c:\Users\ccsar\miniconda3\envs\env_nlp\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=constant, MLP__solver=sgd; total time=10.2min
[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=adaptive, MLP__solver=adam; total time= 7.2min
[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=adaptive, MLP__solver=adam; total time= 7.7min
[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=adaptive, MLP__solver=adam; total time= 5.8min
[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=adaptive, MLP__solver=adam; total time= 7.0min
[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=adaptive, MLP__solver=adam; total time= 7.2min


c:\Users\ccsar\miniconda3\envs\env_nlp\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END MLP__activation=relu, MLP__alpha=0.01, MLP__hidden_layer_sizes=(100, 50), MLP__learning_rate=adaptive, MLP__solver=sgd; total time= 9.9min


In [ ]:
from sklearn.neural_network import MLPClassifier

# Definimos los mejores parámetros encontrados
model_best_params = {
    'hidden_layer_sizes': (50, 50),
    'activation': 'relu',
    'solver': 'sgd',
    'alpha': 0.001,
    'learning_rate': 'constant'
}
model_name = 'MLP'
best_classifier = MLPClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################
Accuracy de MLP: 69.75%


C:\Users\A0860770\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### 2.8 XGBoost

In [ ]:
import xgboost as xgb

# Definimos el modelo a usar
classifier = xgb.XGBClassifier()
model_name = 'XGB'

# Definimos los parámetros a explorar
param_grid = {

    f'{model_name}__max_depth': [3, 5],
    f'{model_name}__learning_rate': [0.1, 0.3],
    f'{model_name}__subsample': [0.8, 1.0],
    f'{model_name}__colsample_bytree': [0.8, 1.0]
}

best_params = train_cv_models(model_name, classifier, param_grid, X_train, y_train)
model_best_params = {k.split('__')[1]:v for k,v in best_params.items() if 'tfidf' not in k}

best_classifier = xgb.XGBClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

######################## Primera búsqueda de parámetros ########################
Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV] END XGB__colsample_bytree=0.8, XGB__learning_rate=0.1, XGB__max_depth=3, XGB__subsample=0.8; total time=   0.1s
[CV] END XGB__colsample_bytree=0.8, XGB__learning_rate=0.1, XGB__max_depth=3, XGB__subsample=0.8; total time=   0.0s
[CV] END XGB__colsample_bytree=0.8, XGB__learning_rate=0.1, XGB__max_depth=3, XGB__subsample=1.0; total time=   0.0s
[CV] END XGB__colsample_bytree=0.8, XGB__learning_rate=0.1, XGB__max_depth=3, XGB__subsample=1.0; total time=   0.0s
[CV] END XGB__colsample_bytree=0.8, XGB__learning_rate=0.1, XGB__max_depth=5, XGB__subsample=0.8; total time=   0.0s
[CV] END XGB__colsample_bytree=0.8, XGB__learning_rate=0.1, XGB__max_depth=5, XGB__subsample=0.8; total time=   0.0s
[CV] END XGB__colsample_bytree=0.8, XGB__learning_rate=0.1, XGB__max_depth=5, XGB__subsample=1.0; total time=   0.0s
[CV] END XGB__colsample_bytree=0.8, XGB

ValueError: 
All the 32 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/imblearn/pipeline.py", line 333, in fit
    self._final_estimator.fit(Xt, yt, **last_step_params["fit"])
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 14

In [ ]:
import xgboost as xgb

# Definimos los mejores parámetros encontrados
model_best_params = {
    'max_depth': 5,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}
model_name = 'XGB'
best_classifier = xgb.XGBClassifier(**model_best_params)
model_stats = train_final_model(model_name, best_classifier, X_train, y_train)
results.append(model_stats)

############### Creando modelo final con los mejores parámetros ################


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44], got [ 1  2  3  4  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29 30 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 48 50]

## 3. Results

In [ ]:
res_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1_score'])
res_df

,Model,Accuracy,Precision,Recall,F1_score
0,LogisticRegression,0.8825,0.883943,0.8825,0.880762
1,NaiveBayes,0.7795,0.796886,0.7795,0.780723
2,KNeighbors,0.6740,0.682722,0.6740,0.673619
3,DecisionTree,0.1885,0.243629,0.1885,0.202258
4,SVC,0.8425,0.860776,0.8425,0.835338
5,RandomForest,0.5570,0.625686,0.5570,0.548560
6,MLP,0.6975,0.712400,0.6975,0.700487


In [ ]:
import winsound
[winsound.Beep(400, 500) for i in range(3)]

[None, None, None]